# Simple attempt at using Qwen2.5-72B for AI Math Olympiad
## (Progress Prize 2)

### Competition provided explanations for prompts:
### https://www.kaggle.com/datasets/richolson/ai-math-olympiad-pp2-reference-solutions

### Credit for Qwen 2.5 Setup Code:
### https://www.kaggle.com/code/abdullahmeda/load-72b-awq-model-using-vllm-on-l4-x4

In [1]:
import os
import sys 
import warnings
import gc
import random
import time

import pandas as pd
import polars as pl

from tqdm import tqdm

import torch 
import kaggle_evaluation.aimo_2_inference_server

pd.set_option('display.max_colwidth', None)

# Load reference problems with solutions
* https://www.kaggle.com/datasets/richolson/ai-math-olympiad-pp2-reference-solutions

In [2]:
reference = pd.read_csv('/kaggle/input/ai-math-olympiad-pp2-reference-solutions/reference_with_solutions.csv')

reference

,id,problem,answer,solution
0,057f8a,"Three airline companies operate flights from Dodola island. Each company has a different schedule of departures. The first company departs every 100 days, the second every 120 days and the third every 150 days. What is the greatest positive integer $d$ for which it is true that there will be $d$ consecutive days without a flight from Dodola island, regardless of the departure times of the various airlines?",79,"The airlines are called A100, A120 and A150, labelled by the frequency of their departures. We first prove that there is a period of 99 days after an A100 departure during which no A150 plane takes off.\n\nConsider a period of 301 days which starts with an A150 departure on Day 0, followed by a departure on Day 150 and on Day 300. Let the first A100 departure in this period be on Day x.\n\nThere are two possibilities: (i) 0 ≤ x ≤ 50 or (ii) 51 ≤ x ≤ 99. In case (i), there is a quiet period of 99 days after the first A100 departure. In case (ii), the second A100 departure will be on Day 100 + x where 151 ≤ 100 + x ≤ 199 so there will be a period of 99 consecutive days after the second A100 departure with no A150 departure.\n\nWe will now prove that there are 79 consecutive days on which no departure of any airline happens, including the A120 planes. We restart time and define Day 0 to be when an A100 flight departs and there is no A150 flight before Day 100. This situation will repeat later because 300 = 3 × 100 = 2 × 150. The fourth A100 flight will take off on Day 300 and there will be no subsequent departure of an A150 plane before Day 400.\n\nSuppose the first departure of an A120 plane is on Day y. If y ≤ 20 or y ≥ 80, we have found the 79 consecutive days by looking after or before this A120 departure.\n\nIf 20 ≤ y ≤ 60, then there will be an A120 departure on day 240 + y where 260 ≤ 240 + y ≤ 300 so there will be no A120 departure strictly between Day 300 and Day 380 and so we will find the required 79 consecutive quiet days between those dates.\n\nFinally, if 61 ≤ y ≤ 80, there will be an A120 departure on Day y+240 where 301 ≤ y+240 ≤ 320 and there will be no subsequent departures before Day 400 and again we find the required 79 consecutive quiet days.\n\nWe now show that this bound can be attained. Suppose that an A100 departs on Day 0, an A120 departs on Day 80 and an A150 departs on Day 120. The departure days are then: 0, 80, 100, 120, 200&200, 270, 300, 320, 400, 420, 440, 500, 560, 570 modulo 600 (i.e. it repeats every 600 days).\n\nThe longest run of consecutive days without flights is 79 days (and this is obtained three times in this 600 day cycle)."
1,192e23,"Fred and George take part in a tennis tournament with $4046$ other players. In each round, the players are paired into $2024$ matches. How many ways are there to arrange the first round such that Fred and George do not have to play each other? (Two arrangements for the first round are \textit{different} if there is a player with a different opponent in the two arrangements.)",250,"Consider an tournament with 2m players. The number of possible first round pairings can be calculated by labelling the matches 1 to m. Label the players in order 1 to 2m (this can be done in (2m)! ways), and assign players (2i - 1) and 2i to match number i.\n\nHow many times does any given particular first round pairing arise from the method specified above? Swapping the labels of any pair 2i - 1, 2i does not change the pairings, nor does permuting the order of the labels of the matches. Doing anything else will result in a different first round pairing, so (2m)! is overcounting the number of first round pairings by a factor of 2^m(m!).\n\nTherefore the number of first round pairings is (2m)!/(2^m m!).\n\nNow consider the odd and even factors in the product defining (2m)! separately. We see that (2m)! = [(2m) · 2(m - 1) · · · 2] × [(2m - 1) · (2m - 3) · · · 3 · 1].\n\nThe product of the odd numbers (2m - 1) · (2m - 3) · · · 3 · 1 is o

# Split our test questions into examples / validate
* Examples are used as part of our prompt
* The remaining questions are tested against

In [3]:
#questions we will use for examples
example_questions = [0, 5]

examples_df = reference.iloc[example_questions]  # Using a list of indices

# Get all other rows for validate
validate_df = reference.drop(example_questions)
validate_df = validate_df.drop(columns=['solution'])


# Save to csv files
examples_df.to_csv('examples.csv', index=False)
validate_df.to_csv('validate.csv', index=False)

# Pick an LLM
* Determines which LLM setup cell gets run
* Placeholder if we want to add other LLMs

In [4]:
class llm_choice:
    qwen_25_72b = 1
    
llm_to_use = llm_choice.qwen_25_72b

#make sure we allow for several hundred words of response...
max_tokens = 1800

# Qwen2.5-72B-Instruct Setup
### https://www.kaggle.com/code/abdullahmeda/load-72b-awq-model-using-vllm-on-l4-x4


In [5]:
if llm_to_use == llm_choice.qwen_25_72b:
    
    from vllm import LLM, SamplingParams
    
    warnings.simplefilter('ignore')

    os.environ["CUDA_VISIBLE_DEVICES"]   = "0,1,2,3"
    os.environ["TOKENIZERS_PARALLELISM"] = "false"

    def clean_memory(deep=False):
        gc.collect()
        if deep:
            ctypes.CDLL("libc.so.6").malloc_trim(0)
        torch.cuda.empty_cache()


    llm_model_pth = '/kaggle/input/qwen2.5/transformers/72b-instruct-awq/1'

    llm = LLM(
        llm_model_pth,
        dtype="half",                # The data type for the model weights and activations
        max_num_seqs=8,              # Maximum number of sequences per iteration. Default is 256
        max_model_len=4096,          # Model context length
        trust_remote_code=True,      # Trust remote code (e.g., from HuggingFace) when downloading the model and tokenizer
        tensor_parallel_size=4,      # The number of GPUs to use for distributed execution with tensor parallelism
        gpu_memory_utilization=0.97, # The ratio (between 0 and 1) of GPU memory to reserve for the model
    )


    sampling_params = SamplingParams(
        temperature=0.3,              # randomness of the sampling
        seed=1,                       # Seed for reprodicibility
        skip_special_tokens=True,     # Whether to skip special tokens in the output.
        max_tokens=max_tokens
    )

    #used to return response for all messages

    def get_response_for_messages(messages):
        response = llm.chat(messages, sampling_params, use_tqdm=False)
        return(response[0].outputs[0].text)

2024-10-27 06:00:05,644	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 10-27 06:00:09 awq_marlin.py:89] The model is convertible to awq_marlin during runtime. Using awq_marlin kernel.
INFO 10-27 06:00:09 config.py:904] Defaulting to use mp for distributed inference
INFO 10-27 06:00:09 llm_engine.py:223] Initializing an LLM engine (v0.6.1.post2) with config: model='/kaggle/input/qwen2.5/transformers/72b-instruct-awq/1', speculative_config=None, tokenizer='/kaggle/input/qwen2.5/transformers/72b-instruct-awq/1', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=4, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=awq_marlin, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_confi

Loading safetensors checkpoint shards:   0% Completed | 0/11 [00:00<?, ?it/s]


(VllmWorkerProcess pid=245) INFO 10-27 06:04:50 model_runner.py:1008] Loading model weights took 9.7897 GB
(VllmWorkerProcess pid=243) INFO 10-27 06:04:50 model_runner.py:1008] Loading model weights took 9.7897 GB
INFO 10-27 06:04:50 model_runner.py:1008] Loading model weights took 9.7897 GB
(VllmWorkerProcess pid=244) INFO 10-27 06:04:50 model_runner.py:1008] Loading model weights took 9.7897 GB
INFO 10-27 06:04:58 distributed_gpu_executor.py:57] # GPU blocks: 9003, # CPU blocks: 3276
INFO 10-27 06:05:01 model_runner.py:1311] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 10-27 06:05:01 model_runner.py:1315] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memo

# Test out the LLM...

In [6]:
messages = [
    {"role": "user", "content": "Tell me about your math skills."},
]

get_response_for_messages(messages)

"I have a robust set of capabilities when it comes to mathematics. I can handle a wide range of tasks, from basic arithmetic to more complex problems involving algebra, calculus, statistics, and even some aspects of advanced mathematics. Here are a few examples of what I can do:\n\n1. **Basic Arithmetic**: I can perform addition, subtraction, multiplication, and division quickly and accurately.\n2. **Algebra**: I can solve linear equations, quadratic equations, and systems of equations. I can also simplify expressions and factor polynomials.\n3. **Calculus**: I can compute derivatives, integrals, and limits. I can also help with differential equations and series expansions.\n4. **Statistics**: I can calculate mean, median, mode, standard deviation, and other statistical measures. I can also help with probability distributions, hypothesis testing, and regression analysis.\n5. **Geometry**: I can solve problems involving shapes, angles, and spatial relationships. I can also help with tri

# Do inference
* Add our examples + explanation to the prompt
* Plead with the LLM to answer only as a number
* Convert to int, modulo 1000

In [7]:
question_suffix = ". Show your work - use at least 100 words - but less than 500 words. You MUST answer in the format of ANSWER: NUMBER.  You MUST conclude each response with ANSWER: NUMBER"

def predict_for_question(question, verbose=False):

    messages = []
    
    #include examples
    for index in range(len(examples_df)):
        messages.append(        
            {
            "role": "user", 
            "content": examples_df.iloc[index].problem + question_suffix
            }
        )
        messages.append(        
            {
            "role": "agent", 
            "content": f"{examples_df.iloc[index].solution} ANSWER: {examples_df.iloc[index].answer}"
            }
        )

    messages.append(
        {
        "role": "user", 
        "content": question + question_suffix
        }
    )
    
    answer = get_response_for_messages(messages)
    
    if verbose:
        print("Prompt:")
        for message in messages:
            print(message)
        print("Response:\n", answer)
        
    #find answer / strip any trailing period
    answer = answer.split("ANSWER: ")[-1].rstrip(".")
    
    # Try to convert the response to an integer and modulo 1000
    try:
        answer = int(answer) % 1000
        return answer
    # if the LLM responds with something we can't convert
    except ValueError:
        # If it fails (e.g., answer isn't a valid number), print error and return 0
        print(f"Error: The answer '{answer}' is not a valid number.")
        return 0
    
# Simple demo...
answer = predict_for_question("What is ((1111*4)/ 2) + 1 + 99", verbose=True)
print("Final answer (mod 1000):", answer)

Prompt:
{'role': 'user', 'content': 'Three airline companies operate flights from Dodola island. Each company has a different schedule of departures. The first company departs every 100 days, the second every 120 days and the third every 150 days. What is the greatest positive integer $d$ for which it is true that there will be $d$ consecutive days without a flight from Dodola island, regardless of the departure times of the various airlines?. Show your work - use at least 100 words - but less than 500 words. You MUST answer in the format of ANSWER: NUMBER.  You MUST conclude each response with ANSWER: NUMBER'}
{'role': 'agent', 'content': 'The airlines are called A100, A120 and A150, labelled by the frequency of their departures. We first prove that there is a period of 99 days after an A100 departure during which no A150 plane takes off.\n\nConsider a period of 301 days which starts with an A150 departure on Day 0, followed by a departure on Day 150 and on Day 300. Let the first A100

# Server Setup

In [8]:
def predict(id_: pl.DataFrame, question: pl.DataFrame, answer: pl.DataFrame | None = None) -> pl.DataFrame | pd.DataFrame:
    """Make a prediction."""
    # Unpack values
    id_ = id_.item(0)
    question = question.item(0)

    # Make a prediction
    prediction = predict_for_question(question)

    # track / print accuracy if not in competition / using train
    if not os.getenv('KAGGLE_IS_COMPETITION_RERUN') :
        global questions_answered
        global verified_correct
        questions_answered += 1
        
        correct_answer = answer.item(0) if answer is not None else None
        if (prediction==correct_answer): verified_correct += 1
        print(f"Prediction: {prediction}, Actual: {correct_answer}, Correct: {prediction==correct_answer}")

    return pl.DataFrame({'id': id_, 'prediction': prediction})

# Start Inference
* Run on validate if not submitting / print score
* Various configurations have been correct for questions with answers of 180, 201 and 143 (but not all at once)

In [9]:
questions_answered = 0
verified_correct = 0

inference_server = kaggle_evaluation.aimo_2_inference_server.AIMO2InferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/kaggle/working/validate.csv',

            # if we want to run on all problems
            #'/kaggle/input/ai-mathematical-olympiad-progress-prize-2/reference.csv',
        )
    )

if not os.getenv('KAGGLE_IS_COMPETITION_RERUN') :
    print("Portion correct: ", verified_correct / questions_answered)

Prediction: 999, Actual: 143, Correct: False
Prediction: 2, Actual: 810, Correct: False
Prediction: 135, Actual: 250, Correct: False
Prediction: 201, Actual: 201, Correct: True
Prediction: 1, Actual: 3, Correct: False
Prediction: 180, Actual: 180, Correct: True
Prediction: 938, Actual: 902, Correct: False
Prediction: 10, Actual: 891, Correct: False
Portion correct:  0.25
